In [ ]:
import whisper
import wave
import pyaudio
import time
import os

In [ ]:
for modelo in ["tiny", "base", "small", "medium", "large"]:
    start = time.time()
    model = whisper.load_model(modelo)
    print(f"Tiempo en cargar el modelo {modelo}: {time.time() - start:.2f}")
    num_params = sum(p.numel() for p in model.parameters())
    model_size = num_params * 4 / (1024 ** 2)
    print(f"El modelo ocupa {model_size:.2f} MBs")
    print(f"Número de parámetros: {num_params}")

In [8]:
def grabar_audio(archivo="comando.wav", duracion=5, tasa_muestreo=16000):
    """Graba audio desde el micrófono y lo guarda en un archivo WAV."""
    formato = pyaudio.paInt16
    canales = 1
    buffer = 1024

    p = pyaudio.PyAudio()
    stream = p.open(format=formato, channels=canales, rate=tasa_muestreo, input=True, frames_per_buffer=buffer)

    print("🎤 Grabando...")

    frames = []
    for _ in range(0, int(tasa_muestreo / buffer * duracion)):
        data = stream.read(buffer)
        frames.append(data)

    print("✅ Grabación finalizada")

    stream.stop_stream()
    stream.close()
    p.terminate()

    with wave.open(archivo, 'wb') as wf:
        wf.setnchannels(canales)
        wf.setsampwidth(p.get_sample_size(formato))
        wf.setframerate(tasa_muestreo)
        wf.writeframes(b''.join(frames))

    return archivo

def transcribir_audio(archivo="comando.wav", modelo="base"):
    """Usa Whisper para transcribir un archivo de audio."""
    print(f"🧠 Cargando modelo {modelo}...")
    model = whisper.load_model(modelo)

    print("🎙️ Transcribiendo audio...")
    resultado = model.transcribe(archivo, language="es")
    
    print("✅ Transcripción completada:")
    print(resultado["text"])
    
    return resultado["text"]

In [18]:
archivo = grabar_audio(duracion=20)

🎤 Grabando...
✅ Grabación finalizada


In [19]:
folder_path = os.path.join(os.getcwd(), "data", "Demo2")
os.makedirs(folder_path, exist_ok=True)

for modelo in ["tiny", "base", "small", "medium", "large"]:
    inicio = time.time()
    print(f"Probando modelo {modelo}")
    transcript = transcribir_audio("comando.wav", modelo)

    file_path = os.path.join(folder_path, f"Transcripcion1_{modelo}.txt")

    with open(file_path, "w", encoding="utf-8") as file:
        file.write(transcript)
    print(f"Tiempo de ejecución: {time.time() - inicio:.2f} s")

Probando modelo tiny
🧠 Cargando modelo tiny...
🎙️ Transcribiendo audio...
✅ Transcripción completada:
 Abris Fits de paciente. Javier Rios Montes, de 21 años de edad, nacido el 26 de diciembre de 2003 con número de teléfono 6, 5, 9, 1, 5, 5, 5, 2, 4. Se queja de que le dule las muelas. Tras una inspección preliminar, vamos a arrancarle una una todas las muelas.
Tiempo de ejecución: 1.77 s
Probando modelo base
🧠 Cargando modelo base...
🎙️ Transcribiendo audio...
✅ Transcripción completada:
 Abrié el ficha de paciente. Javier Rios Montes, de 21 años de edad nacido el 26 de diciembre de 2003, con número de teléfono 659-155-524. Se queja de que le dole las muelas. Tras una inspección preliminar vamos a arrancarle una a una todas las muelas.
Tiempo de ejecución: 2.53 s
Probando modelo small
🧠 Cargando modelo small...
🎙️ Transcribiendo audio...
✅ Transcripción completada:
 Abre ficha de paciente. Javier Rios Montés, de 21 años de edad, nacido el 26 de diciembre de 2003 con número de teléfono